In [47]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer
import torch

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [49]:
model = GPT2LMHeadModel.from_pretrained("gunwoo723/kogpt2-generate-poem").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("gunwoo723/kogpt2-generate-poem",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [50]:
tokenizer.add_special_tokens({'additional_special_tokens': ["<yun>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(51201, 768)

In [51]:
sentence = '<s>멈춰버린 시계 속 마지막으로 남은 바램\n'
input_ids = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        temperature=0.5, # 생성 다양성 조절
        min_length=128, # 생성되는 문장의 최소 길이
        max_length=512, # 생성되는 문장의 최대 길이
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        early_stopping=True, # EOS token을 만나면 조기 종료
        eos_token_id=tokenizer.eos_token_id
    )

generated_sequence = output[0].tolist()
text = tokenizer.decode(generated_sequence, skip_special_tokens=False)

In [52]:
text = text.replace("<yun> ", "\n").replace("<s> ", "").replace("</s>", "")

In [53]:
print(text)

멈춰버린 시계 속 마지막으로 남은 바램
하얀 목련이 피던 그날처럼 
내 마음도 한겨울의 찬바람에 녹아버렸네

내 그대를 향한 고백은 끝이 없지만
그대와의 만남을 위해선 기다림이 필요해
하루 하루가 다르게 자라나는 나만의 추억들

겨울 산길에서 만난 그대 향기
가슴깊이 묻어 둔 사연 들추어내어
고운 눈빛으로 그려보는 나의 사랑아

떠나간다는 것은 새봄의 환희로 다가온다지만
언제나 변함없는 내 모습 그리워
나만 그런 것이 아니라면 괜찮겠지
